This Notebook contains the required code to setup and query data from the local SQL database. It uses the SQLite3 library. The tables follow the structure desribed in the README file.

In [2]:
import sqlite3 as sq
import json
from time_entries_controller import TimeCampEntry
from debug_logger import setup_debug_logger
logger = setup_debug_logger()

Using only sqlite3 basic fonctionalities, running SQL instructions.

In [3]:
def create_connection() -> sq.Connection:
    try:
        conn = sq.connect('time_tracker.db')
    except sq.Error as e:
        logger.error(f'Connection creation got error: {e}')
    finally:
        return conn
    
def close_connection(conn: sq.Connection) -> None:
    try:
        conn.close()
    except sq.Error as e:
        logger.error(f'Connection closing got error: {e}')

Creating the Users table

In [4]:
def create_users_table() -> None:
    """
    Creates a new table called 'Users' in the database with the following structure:
    
    - UserID: INTEGER (Primary Key)
    - Name: TEXT (Not Null)
    - Email: TEXT (Not Null, Unique)
    - Token: TEXT (Not Null, Unique)
    """
    conn = create_connection()
    try:
        c = conn.cursor()
        # Drop the existing table if it exists
        c.execute('DROP TABLE IF EXISTS Users')
        
        # Create the new table with the desired structure
        c.execute('''
        CREATE TABLE Users (
            UserID INTEGER PRIMARY KEY,
            Name TEXT NOT NULL,
            Email TEXT NOT NULL UNIQUE,
            Token TEXT NOT NULL UNIQUE
        )
        ''')
        conn.commit()
    except sq.Error as e:
        logger.error(f'Creating users table got error: {e}')
    finally:
        close_connection(conn)

create_users_table()


Creating the TimeEntries table

In [5]:
def create_time_entries_table() -> None:
    conn = create_connection()
    try:
        c = conn.cursor()
        # Drop the existing table if it exists
        c.execute('DROP TABLE IF EXISTS TimeEntries')
        # Create the new table with the desired structure
        c.execute('''
            CREATE TABLE TimeEntries (
            TimeEntryID TEXT PRIMARY KEY,
            UserID TEXT,
            Date TEXT NOT NULL,
            StartTime TEXT NOT NULL,
            EndTime TEXT NOT NULL,
            Duration TEXT,
            TaskID TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID)
        )
        ''')
        # The foreign key is the link between the two tables. It ensures that the UserID in the TimeEntries table is a valid UserID in the Users table.
    except sq.Error as e:
        logger.error(f'Creating time_entries table got error: {e}')
    finally:
        close_connection(conn)
create_time_entries_table()

In [22]:
# Add entry to database
def insert_time_entry(time_entry: TimeCampEntry) -> None:
    conn = create_connection()
    try:
        c = conn.cursor()
        c.execute(f"INSERT INTO TimeEntries (TimeEntryID, UserID, Date, StartTime, EndTime, Duration, TaskID) VALUES (?, ?, ?, ?, ?, ?, ?)", 
                  (time_entry.id, time_entry.user_id, time_entry.date, time_entry.start_time, time_entry.end_time, time_entry.duration, time_entry.task_id))
        conn.commit()
    except sq.Error as e:
        logger.error(f'Inserting time entry {time_entry.id} into local database got: {e}')
    finally:
        close_connection(conn)


with open('fake_time_entry_content.json') as f:
    fake_time_entry = TimeCampEntry(json.load(f))
print(f'Fake time entry data: {fake_time_entry.print_attributes()}')
insert_time_entry(fake_time_entry)

def query_time_entries() -> None:
    conn = create_connection()
    try:
        c = conn.cursor()
        c.execute('SELECT * FROM TimeEntries')
        rows = c.fetchall()
        for row in rows:
            print(row)
    except sq.Error as e:
        logger.error(f'Querying time entries got error: {e}')
    finally:
        close_connection(conn)

query_time_entries()



TimeCampEntry Class Attributes
id : 199699361
duration : 1320
user_id : 2056041
user_name : david.knabenhans@gmail.com
task_id : 149709760
task_note : 
last_modify : 2024-02-12 08:11:31
date : 2023-10-18
start_time : 20:00:00
end_time : 20:22:00
locked : 0
name : 2023_Bob
addons_external_id : board_63ca7d8b28bd870141457e75
billable : 1
invoice_id : 0
color : #34C644
description : FAKE TIME ENTRY
Fake time entry data: None
('1234567', None, '2021-12-23', '11:00:00', '12:00:00', '60', '789')
('19912', None, '2023-10-18', '20:00:00', '20:22:00', '1320', '149709760')
('199699361', '2056041', '2023-10-18', '20:00:00', '20:22:00', '1320', '149709760')
